In [45]:
import pandas as pd
import numpy as np
from numpy import unique
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

In [28]:
#----------concatanating the data

data = pd.read_csv('../data/merged_patient0.csv')
for i in range(1,40):
    data = pd.concat([data, pd.read_csv('../data/merged_patient'+str(i)+'.csv')],ignore_index=True)

print('shape of the dataset...',data.shape)

#-----categorical data encoding ---------
cat = data[data.columns[-6-1:-1]]
print('number of categorical columns...',cat.shape[1])
encode = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame((encode.fit_transform(cat)).toarray())
enc_df.columns = ['RTG_h','RTG_l','RTG_n','RF_n','liver_i','liver_n','liver_r','peripheral_i','peripheral_n','peripheral_r','par1B', 'par1C', 'par1E', 'par1x','par2a', 'par2b', 'par2g', 'par2h', 'par2j', 'par2k', 'par2l', 'par2n', 'par2o', 'par2s', 'par2u', 'par2x']

###-----restructing the dataset-------
y=data['expected_glucose']
data=data.drop(data.columns[-1],axis=1)
data=data.drop(data.columns[-6:],axis=1)
data=data.drop(data.columns[0],axis=1)
data = data.join(enc_df)
data=data.join(y)
dataset=np.asarray(data,dtype='float32')

#---------- split into train and test sets
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
trainX,trainY = train[:,:-1],train[:,-1]
testX,testY = test[:,:-1],test[:,-1]

#----------shapes of the training and the testing datsets
print("shape of training data...",trainX.shape)
print("shape of testing data...",testX.shape)

shape of the dataset... (3880, 33)
number of categorical columns... 6
shape of training data... (2716, 51)
shape of testing data... (1164, 51)


In [58]:
#--training the model---
model=Sequential()
model.add(Dense(4,activation='relu',input_shape=(51,)))
model.add(Dense(2,activation='relu'))
model.add(Dense(1))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
hist=model.fit(x=trainX,y=trainY,validation_data=[testX,testY], epochs=30)


Epoch 1/30
68/85 [=======================>......] - ETA: 0s - loss: -125.3894 - accuracy: 0.0000e+00

ValueError: in user code:

    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:909 test_step  **
        y_pred = self(x, training=False)
    /home/dipanshu/qsp_project/lib/python3.8/site-packages/tensorflow/python/k

In [55]:
? Dense